In [1]:
import pandas as pd

In [3]:
melbourne_data=pd.read_csv("melb_data.csv")
melbourne_data.describe()

,Rooms,Price,Distance,Postcode,Bedroom2,Bathroom,Car,Landsize,BuildingArea,YearBuilt,Lattitude,Longtitude,Propertycount
count,13580.000000,1.358000e+04,13580.000000,13580.000000,13580.000000,13580.000000,13518.000000,13580.000000,7130.000000,8205.000000,13580.000000,13580.000000,13580.000000
mean,2.937997,1.075684e+06,10.137776,3105.301915,2.914728,1.534242,1.610075,558.416127,151.967650,1964.684217,-37.809203,144.995216,7454.417378
std,0.955748,6.393107e+05,5.868725,90.676964,0.965921,0.691712,0.962634,3990.669241,541.014538,37.273762,0.079260,0.103916,4378.581772
min,1.000000,8.500000e+04,0.000000,3000.000000,0.000000,0.000000,0.000000,0.000000,0.000000,1196.000000,-38.182550,144.431810,249.000000
25%,2.000000,6.500000e+05,6.100000,3044.000000,2.000000,1.000000,1.000000,177.000000,93.000000,1940.000000,-37.856822,144.929600,4380.000000
50%,3.000000,9.030000e+05,9.200000,3084.000000,3.000000,1.000000,2.000000,440.000000,126.000000,1970.000000,-37.802355,145.000100,6555.000000
75%,3.000000,1.330000e+06,13.000000,3148.000000,3.000000,2.000000,2.000000,651.000000,174.000000,1999.000000,-37.756400,145.058305,10331.000000
max,10.000000,9.000000e+06,48.100000,3977.000000,20.000000,8.000000,10.000000,433014.000000,44515.000000,2018.000000,-37.408530,145.526350,21650.000000


In [17]:
y=melbourne_data.Price
X=melbourne_data.drop("Price",axis=1)

In [32]:
from sklearn.model_selection import train_test_split
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import OneHotEncoder
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.metrics import mean_absolute_error
from sklearn.ensemble import RandomForestRegressor
X_train,X_valid,y_train,y_valid=train_test_split(X,y,train_size=0.8,test_size=0.2,random_state=0)

numerical_cols=[col for col in X_train.columns if X_train[col].dtype in ['int64','float64']]
print("Numerical cols:")
print(numerical_cols)
categorical_cols=[col for col in X_train.columns if X_train[col].dtype == 'object' and X_train[col].nunique()<10]
print("Categorical cols:")
print(categorical_cols)
my_cols=numerical_cols+categorical_cols
X_train=X_train[my_cols].copy()
X_valid=X_valid[my_cols].copy()

Numerical cols:
['Rooms', 'Distance', 'Postcode', 'Bedroom2', 'Bathroom', 'Car', 'Landsize', 'BuildingArea', 'YearBuilt', 'Lattitude', 'Longtitude', 'Propertycount']
Categorical cols:
['Type', 'Method', 'Regionname']


In [46]:
numerical_transformer=SimpleImputer(strategy='constant')
categorical_transformer=Pipeline(steps=[
    ('imputer',SimpleImputer(strategy='most_frequent')),
    ('onehot',OneHotEncoder(handle_unknown='ignore'))
])
preprocessor=ColumnTransformer(
    transformers=[
        ('num',numerical_transformer,numerical_cols),
        ('cat',categorical_transformer,categorical_cols)
    ]
)
model=RandomForestRegressor(n_estimators=100,random_state=1)
my_pipeline=Pipeline(steps=[
    ('preprocessor',preprocessor),
    ('model',model)
])
from sklearn.model_selection import cross_val_score
scores=-1*cross_val_score(my_pipeline,X_train,y_train,cv=5,scoring='neg_mean_absolute_error')
print("Calculating score:")
print(scores)
print("Calculating mean score:")
print(scores.mean())

Calculating score:
[166499.33723358 166240.47589398 163265.28360682 165046.60090373
 170316.58829168]
Calculating mean score:
166273.6571859581
